#DataPreprocessing and Final Dataframe production


## Importing the required modules


In [ ]:
import pandas as pd

##Reading the required csv files one of the responses and other of character traits of the user.




In [ ]:
df = pd.read_csv('responses.csv')

In [ ]:
physical_appearance = pd.read_csv('personality3.csv')

In [ ]:
final_personality = pd.read_csv('final_seperate_personality.csv')

##Fine-tuning data 

In [ ]:
df = df.drop(['Timestamp'],axis=1)
df = df.replace('COE,CSE,CSBS','COE')
df = df.replace('ENC,EEC,ECE and similar','ECE')
df = df.replace('ENC,EEC,ECE','ECE')
df = df.replace('BME,BT','BME')
df = df.replace('MEC,CIE','MEC')

In [ ]:
df.rename(columns = {'Select your favourite movie from below.':'Movie'}, inplace = True)

In [ ]:
df.insert(9, "Characters", '0', True)

In [ ]:
characters = df.iloc[:,2:]

In [ ]:
for i in range(characters.shape[0]):
  for column in characters.drop(['Characters'],axis=1).columns:
    if str(characters.iloc[i][column]) != "nan":
      characters.iloc[i]['Characters'] = characters.iloc[i][column]
      break

In [ ]:
final_characters = characters['Characters']
for k in characters.columns:
  df = df.drop([k],axis=1)
df['Characters'] = final_characters

In [ ]:
final_personality = final_personality.fillna('none')

##Assigning appearances to each entry depending on the character(s) chosen

In [ ]:
appearance = []
for characters in df['Characters']:
  if len(characters.split(';'))>1:
    temp = []
    for character in characters.split(';'):
      character = character.strip()
      temp.append(physical_appearance.iloc[list(physical_appearance['Character']).index(character)][1])
    final_text = " "
    for k in temp:
      for word in k.split(','):
        final_text += word.strip() + ','
    appearance.append(final_text[:-1])
  else:
    for character in characters.split(';'):
      character = character.strip()
      appearance.append(physical_appearance.iloc[list(physical_appearance['Character']).index(character)][1])

In [ ]:
df['Appearance']=appearance

In [ ]:
df

,Branch,Movie,Characters,Appearance
0,COE,Zootopia,Chief Bogo,"grey skin, black eye,wear glasses"
1,COE,Hotel Transylvania,Mavis;Jonathan;Frankenstien;Dracula,"fair skin,blue eyes,short black hair,fair ski..."
2,ECE,Inside Out,Anger,"crimson skin, black eyes, red hair"
3,ECE,Up,Carl Fredricksen;Russel;Dug,"fair skin,angular glasses,black eyes,white ha..."
4,COE,Inside Out,Riley;Joy;Sadness,"fair skin,blue eyes,dirty blonde hair,yellow ..."
...,...,...,...,...
130,CHE,Shrek,Shrek;Donkey,"green skin,horns,balck eyes,grey skin,black m..."
131,CHE,Shrek,Shrek;Donkey,"green skin,horns,balck eyes,grey skin,black m..."
132,CHE,Shrek,Shrek;Donkey,"green skin,horns,balck eyes,grey skin,black m..."
133,CHE,Shrek,Shrek;Donkey,"green skin,horns,balck eyes,grey skin,black m..."


#Processing on DataFrame

In [ ]:
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def getCharacter(sentence):
  similarity = 0
  character = ''
  for k in list(physical_appearance['Appearance']):
    X =sentence
    Y =k
    X_list = word_tokenize(X)
    Y_list = word_tokenize(Y)
    sw = stopwords.words('english')
    l1 =[];l2 =[]
    X_set = {w for w in X_list if not w in sw}
    Y_set = {w for w in Y_list if not w in sw}
    rvector = X_set.union(Y_set)
    for w in rvector:
      if w in X_set: l1.append(1)
      else: l1.append(0)
      if w in Y_set: l2.append(1)
      else: l2.append(0)
    c = 0
    for i in range(len(rvector)):
        c+= l1[i]*l2[i]
    cosine = c / float((sum(l1)*sum(l2))**0.5)
    if(similarity<cosine):
      similarity = cosine
      character = physical_appearance.iloc[list(physical_appearance['Appearance']).index(k)][0]
  return character

In [ ]:
predicted_characters = []
for i in range(df.shape[0]):
  predicted_characters.append(getCharacter(df.iloc[i][3]))

In [ ]:
df['Predicted Characters'] = predicted_characters

In [ ]:
correct=0
wrong=0
for i in range(df.shape[0]):
  if(df.iloc[i][4] in df.iloc[i][2]):
    correct+=1
  else:
    wrong+=1
print('Correct:',correct)
print('Wrong:',wrong)

Correct: 125
Wrong: 10


In [ ]:
model_accuracy = correct/(correct+wrong)
model_accuracy

0.9259259259259259

In [ ]:
temp = list(df['Branch'])
temp = set(temp)

In [ ]:
records = []
coe = pd.DataFrame()
ece = pd.DataFrame()
che = pd.DataFrame()
bme = pd.DataFrame()
mec = pd.DataFrame()
for branch in temp:
  for i in range(df.shape[0]):
    if(branch == df.loc[i][0]):
      records.append(df.loc[i])
      if (branch == 'COE'):
        coe = pd.DataFrame(records)
      elif (branch == 'ECE'):
        ece = pd.DataFrame(records)
      elif (branch == 'CHE'):
        che = pd.DataFrame(records)
      elif (branch == 'BME'):
        bme = pd.DataFrame(records)
      elif (branch == 'MEC'):
        mec = pd.DataFrame(records)
  records=[]

In [ ]:
coe = coe.drop(['Branch'],axis=1)
mec = mec.drop(['Branch'],axis=1)
ece = ece.drop(['Branch'],axis=1)
che = che.drop(['Branch'],axis=1)
bme = bme.drop(['Branch'],axis=1)

#Branch-wise Output

In [ ]:
def getlnk(name,skin,hat,glasses,eye,hair,misc):
  url = 'https://model-results.herokuapp.com/name='+name+'&skincolor='+skin+'&hat='+hat+'&glasses='+glasses+'&eye='+eye+'&misc='+misc+'&hair='+hair
  return url

In [ ]:
def getfinallnk(character):
  for i in range(final_personality.shape[0]):
    if final_personality.iloc[i][0] == character:
      return getlnk(character.lower(),final_personality.iloc[i][1].lower(),final_personality.iloc[i][2].lower(),final_personality.iloc[i][3].lower(),final_personality.iloc[i][4].lower(),final_personality.iloc[i][5].lower(),final_personality.iloc[i][6].lower())

In [ ]:
pred = list(coe['Predicted Characters'])
unique = list(set(pred))
count = []
for character in unique:
  count.append(pred.count(character))
final_coe_char = unique[count.index(max(count))]
print("The character assigned to COE is:",final_coe_char)
print(getfinallnk(final_coe_char))

The character assigned to COE is: Hiro Hamada
https://model-results.herokuapp.com/name=hiro hamada&skincolor=fair&hat=none&glasses=none&eye=brown &misc=blue hoodie jacket, red shirt,&hair=black


In [ ]:
pred = list(mec['Predicted Characters'])
unique = list(set(pred))
count = []
for character in unique:
  count.append(pred.count(character))
print("The character assigned to MEC is:",unique[count.index(max(count))])
print(getfinallnk(unique[count.index(max(count))]))

The character assigned to MEC is: Disgust
https://model-results.herokuapp.com/name=disgust&skincolor=green &hat=none&glasses=none&eye=green &misc=spring green dress without sleeves, pink scarf tied around her neck,&hair=green


In [ ]:
pred = list(ece['Predicted Characters'])
unique = list(set(pred))
count = []
for character in unique:
  count.append(pred.count(character))
print("The character assigned to ECE is:",unique[count.index(max(count))])
print(getfinallnk(unique[count.index(max(count))]))

The character assigned to ECE is: Joy
https://model-results.herokuapp.com/name=joy&skincolor=yellow&hat=none&glasses=none&eye=blue&misc= lime green dress, speckled with blue firework-style flowers&hair=blue


In [ ]:
pred = list(che['Predicted Characters'])
unique = list(set(pred))
count = []
for character in unique:
  count.append(pred.count(character))
print("The character assigned to CHE is:",unique[count.index(max(count))])
print(getfinallnk(unique[count.index(max(count))]))

The character assigned to CHE is: Donkey
https://model-results.herokuapp.com/name=donkey&skincolor=grey&hat=black mane&glasses=none&eye=brown &misc=none&hair=black


In [ ]:
pred = list(bme['Predicted Characters'])
unique = list(set(pred))
count = []
for character in unique:
  count.append(pred.count(character))
print("The character assigned to BME is:",unique[count.index(max(count))])
print(getfinallnk(unique[count.index(max(count))]))

The character assigned to BME is: Nick Wilde
https://model-results.herokuapp.com/name=nick wilde&skincolor=red&hat=none&glasses=none&eye=green&misc=light green hawaiian shirt, dark indigo necktie with red and blue stripes, grayish-tan pants&hair=orange
